In [31]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import FunctionTransformer
from pandas import DataFrame, Series
from sklearn.feature_selection import RFE, RFECV, SelectFromModel,SelectKBest, f_classif
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV, ridge_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import platform
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


In [32]:
X = pd.read_csv('../project1/final_features.csv',index_col=0 )
y = pd.read_csv('../project1/y_open_0910.csv')

X.columns

Index(['date', 'Dividend_Yield', 'sp500_closing', 'PER', 'kodex_open',
       'kodex_volume', 'KR_10Y'],
      dtype='object')

In [33]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

# 날짜 변환 함수 정의
def convert_to_datetime(series):
    return series.apply(lambda x: pd.to_datetime(x, format="%Y-%m-%d"))

# 데이터 타입 변환 
# def convert_to_datatype(series):
#    return series.apply(lambda x: x.astype(int))

# 시프트 함수 정의
def shift_column(series):
    return series.shift(-1)

# Pipeline 및 ColumnTransformer 생성
preprocessor = ColumnTransformer(
    transformers=[
        ('date_transform', FunctionTransformer(func=convert_to_datetime, validate=False), ['date']),
        # ('convert_to_datatype', FunctionTransformer(func=convert_to_datatype, validate=False), ['kodex_open']),  # Fixed function name
        ('shift_columns', FunctionTransformer(func=shift_column, validate=False), ['open', 'kodex_open']),
        ('imputer', SimpleImputer(strategy='mean'), ['Dividend_Yield', 'sp500_closing']),
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# Pipeline을 사용하여 데이터 전처리 수행
df_transformed = pd.merge(X.join(y), pd.DataFrame(pipeline.fit_transform(X.join(y)), columns=['date', 'feature1', 'feature2', 'kodex_open', 'open']), on='date', how='inner')

# 변환된 데이터프레임 출력
df_transformed


ValueError: columns overlap but no suffix specified: Index(['date'], dtype='object')

In [ ]:

# Assuming X and y are your feature matrix and target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Corrected pipeline variable name
my_pipeline = Pipeline([
    ('Scalling', StandardScaler()),
    ('Feature_Selection', SelectKBest(f_classif, k=2)),
    ('RandomForestClassifier', RandomForestClassifier(max_depth=3))])

my_pipeline.fit(X_train)
predictions = my_pipeline.predict(X_test)[:3]
accuracy = my_pipeline.score(X_test, y_test)

print("Predictions:", predictions)
print("Accuracy:", accuracy)


ValueError: Found input variables with inconsistent numbers of samples: [1363, 1241]

In [ ]:
# Feature selection 

models = {'Lasso': Lasso()}
         #'Ridge': ridge_regression()}

feature = RFE(LinearRegression(), n_features_to_select=5)

pipe = Pipeline([
    ('feature_select',feature),
    ('model', models['Lasso'])])

param = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}
         #Ridge_params = 'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}
    
grid = GridSearchCV(estimator=pipe, param_grid=param, cv=3)
grid.fit(X_train,y_train)

NameError: name 'X_train' is not defined

In [ ]:
# 


def model_basic(X_train, y_train, X_test, y_test):
    models = [
        LogisticRegression(),
        SVC(),
        RandomForestRegressor(),
        XGBRegressor(),
        LGBMRegressor()
    ]

    rdict={'model':[],'MSE':[],'MAE':[],'r2_score':[]}

    def model_basic(X_train, y_train, X_test, y_test):
    models = [
        ('Logistic Regression', LogisticRegression()),
        ('SVC', SVC()),
        ('Random Forest Regressor', RandomForestRegressor()),
        ('XGB Regressor', XGBRegressor()),
        ('LGBM Regressor', LGBMRegressor())
    ]

    rdict = {'model': [], 'MSE': [], 'MAE': [], 'r2_score': []}

    for name, clf in models:
        pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),  # Use StandardScaler for numerical features
            ('classifier', clf)
        ])

        # Fit the pipeline
        pipeline.fit(X_train, y_train)

        # Make predictions
        pred = pipeline.predict(X_test)

        # Calculate metrics
        results = (
            round(mean_squared_error(y_test, pred), 2),
            round(mean_absolute_error(y_test, pred), 2),
            round(r2_score(y_test, pred), 2)
        )

        rdict['model'].append(name)
        rdict['MSE'].append(results[0])
        rdict['MAE'].append(results[1])
        rdict['r2_score'].append(results[2])

    rdf = pd.DataFrame(data=rdict)
    return rdf



In [ ]:
p_model = Pipeline(memory=None,
                   steps=[('poly',PolynomialFeatures(degree=2,include_bias=False, interaction_only=False,order='C')),
                    ('Linear',LinearRegression(copy_X=True,fit_intercept=True,n_jobs=None))]
                    ,verbose=False)